# Modding Minecraft
## Modding Python

## Installation

Go to http://www.wiley.com/WileyCDA/Section/id-823690.html and download the **Starter Kit** for your platform (I used the Mac Starter Kit to run on Linux).  The **readme.txt** included will walk you through getting up and running.  It looks like **Bukkit** is now a defunct project, so I used **CanaryMod**.

The major components you will need:

* CanaryMod (http://canarymod.net/releases)
* RaspberryJuice (https://github.com/martinohanlon/canaryraspberryjuice)
* Minecraft (https://minecraft.net)

Checkout the code from this presentation (https://github.com/tobynance/okcpython.git)
> cd okcpython/presentation/2015_07_08_modding_minecraft/code

### Make Sure To...
Open the eula.txt file in the Canarymod folder and change:
> eula=false

to:

> eula=true

Now Run CanaryMod:
> java -Xmx1024M -jar Canarymod.jar

And execute the server command:
> playermod group add `<playername>` admins

## Running

After you have followed the installation steps in the **readme.txt** of the Starter Kit:

### Start CanaryMod
> java -Xmx1024M -jar Canarymod.jar

### Connect to your local CanaryMod server
  + In Minecraft, choose **Multiplayer**, click Add server
  + For the **Server Name** I used *`Canary`*
  + For the **Server Address** put *`localhost`*
  + Click **Done**
  
![Add Server](minecraft_adding_server.png)
  

You should now have the Canary server showing in the Server List

![Server List](minecraft_server_list.png)

## Finally, some code

### Floor of Dirt

In [6]:
from mcpi import block
from mcpi import minecraft

########################################################################
def floor_of_dirt():
    world = minecraft.Minecraft.create()
    player = world.player

    pos = player.getTilePos()
    width = 10
    world.setBlocks(pos.x-width,
                    pos.y-1,
                    pos.z-width,
                    pos.x+width,
                    pos.y-1,
                    pos.z+width,
                    block.DIRT.id)

########################################################################
floor_of_dirt()

### Safe Walk (02_safe_walk.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block

########################################################################
class Mod(BaseCommand):
    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        pos.y -= 1  # block BELOW the player
        self.set_block(pos, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()

### Safe Walk continued (03_safe_walk_platform.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]


########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.01

    ####################################################################
    def get_surrounding_blocks(self, pos, distance=1):
        """
        Get all the positions around `pos` on the XZ plane,
        including the `pos` block.
        """
        for x in range(-distance, distance+1):
            for z in range(-distance, distance+1):
                yield Vec3(pos.x+x, pos.y, pos.z+z)

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        pos.y -= 1  # block BELOW the player

        for p in self.get_surrounding_blocks(pos):
            b = self.get_block(p)
            if b in REPLACE_BLOCKS:
                self.set_block(p, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()

### Safe Walk continued (04_safe_walk_faster_platform.py and base_command.py)

In [ ]:
# in base_command.BaseCommand

    ####################################################################
    def get_blocks(self, start_pos, end_pos):
        args = list(start_pos) + list(end_pos)
        # print "get_blocks:", args
        blocks = self.world.getBlocks(*args)
        block_index = 0
        for x in range(start_pos.x, end_pos.x+1):
            for y in range(start_pos.y, end_pos.y+1):
                for z in range(start_pos.z, end_pos.z+1):
                    yield (Vec3(x, y, z), blocks[block_index])
                    block_index += 1

In [ ]:
# 04_safe_walk_faster_platform.py
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]


########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.05

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        blocks = self.get_blocks(Vec3(pos.x-1,
                                      pos.y-2,
                                      pos.z-1),
                                 Vec3(pos.x+1,
                                      pos.y-1,
                                      pos.z+1))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.set_block(p, block.ICE.id)

########################################################################
if __name__ == "__main__":
    Mod().run()


### Safe Walk continued (05_safe_walk_faster_platform.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.AIR.id]

########################################################################
class Mod(BaseCommand):
    heart_beat_rate = 0.05

    ####################################################################
    def __init__(self):
        super(Mod, self).__init__()
        self.old_ice_blocks = set()

    ####################################################################
    def on_heart_beat(self):
        pos = self.player.getTilePos()
        for p, b in self.old_ice_blocks.copy():
            if p.y >= pos.y or (pos - p).length() > 4:
                self.set_block(p, b)
                self.old_ice_blocks.remove((p, b))

        blocks = self.get_blocks(Vec3(pos.x-2,
                                      pos.y-2,
                                      pos.z-2),
                                 Vec3(pos.x+2,
                                      pos.y-1,
                                      pos.z+2))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.old_ice_blocks.add((p, b))
                self.set_block(p, block.ICE.id)


########################################################################
if __name__ == "__main__":
    Mod().run()

### Skating Rink (06_skating_rink.py)

In [ ]:
from base_command import BaseCommand
from mcpi import block
from mcpi.vec3 import Vec3

REPLACE_BLOCKS = [block.WATER.id,
                  block.WATER_FLOWING.id,
                  block.WATER_STATIONARY.id,
                  block.ICE.id]

RANGE = 5

########################################################################
class Mod(BaseCommand):
    ####################################################################
    def once(self):
        self.replaced = set()
        self.to_replace = set()
        pos = self.player.getTilePos()

        blocks = self.get_blocks(Vec3(pos.x-RANGE,
                                      pos.y-1,
                                      pos.z-RANGE),
                                 Vec3(pos.x+RANGE,
                                      pos.y-1,
                                      pos.z+RANGE))
        for p, b in blocks:
            if b in REPLACE_BLOCKS:
                self.to_replace.add(p)
        self.make_rink()

    ####################################################################
    def make_rink(self):
        while len(self.to_replace) > 0:
            pos = self.to_replace.pop()
            print "replacing block at", pos
            self.set_block(pos, block.ICE.id)
            self.replaced.add(pos)

            # check all neighbors
            for x in [-1, 0, 1]:
                for z in [-1, 0, 1]:
                    if x == z == 0:
                        continue
                    new_pos = Vec3(pos.x+x,
                                   pos.y,
                                   pos.z+z)
                    if new_pos in self.replaced or new_pos in self.to_replace:
                        continue
                    b = self.get_block(new_pos)
                    if b in REPLACE_BLOCKS:
                        self.to_replace.add(new_pos)

########################################################################
if __name__ == "__main__":
    Mod().once()